<a href="https://colab.research.google.com/github/lkhok22/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting/blob/main/model_experiment_SARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Enhanced SARIMAX Model for Walmart Sales Forecasting
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import time
import wandb
import warnings
warnings.filterwarnings("ignore")

In [2]:
# ✅ Only install what you need for SARIMAX and logging
!pip install statsmodels wandb --quiet

In [21]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import wandb
import os
import gc
import psutil
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import wandb
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn scikit-learn torch torchvision wandb pyyaml darts --quiet
import wandb
wandb.login(key="eccf2c915699fc032ad678daf0fd4b5ac60bf87c")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abakh22 (abakh22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
# Mount Google Drive and extract data
from google.colab import drive
import zipfile
import os
drive.mount('/content/drive')
zip_path = '/content/drive/MyDrive/ML-FinalProject/data.zip'
extract_to = '/content/walmart_data/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
for file_name in os.listdir(extract_to):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(extract_to, file_name), 'r') as zip_ref:
            zip_ref.extractall(extract_to)
print("✅ Extracted files:", os.listdir(extract_to))

Mounted at /content/drive
✅ Extracted files: ['test.csv.zip', 'stores.csv', 'sampleSubmission.csv', 'features.csv', 'test.csv', 'train.csv.zip', 'train.csv', 'sampleSubmission.csv.zip', 'features.csv.zip']


In [17]:
# ✅ Memory-optimized preprocessing with reduced dtypes
dtypes = {
    'Store': 'int8', 'Dept': 'int8', 'Weekly_Sales': 'float32',
    'Temperature': 'float32', 'Fuel_Price': 'float32', 'CPI': 'float32',
    'Unemployment': 'float32', 'Size': 'int32', 'IsHoliday': 'bool',
    'MarkDown1': 'float32', 'MarkDown2': 'float32', 'MarkDown3': 'float32',
    'MarkDown4': 'float32', 'MarkDown5': 'float32'
}

# Load data
train = pd.read_csv('/content/walmart_data/train.csv', dtype=dtypes, parse_dates=['Date'])
features = pd.read_csv('/content/walmart_data/features.csv', dtype=dtypes, parse_dates=['Date'])
stores = pd.read_csv('/content/walmart_data/stores.csv', dtype={'Store': 'int8', 'Size': 'int32'})
test = pd.read_csv('/content/walmart_data/test.csv', dtype=dtypes, parse_dates=['Date'])

# Merge and keep necessary columns
df = pd.merge(train, features, on=['Store', 'Date'], how='left')
df = pd.merge(df, stores, on='Store', how='left')
df = df.drop(columns=['IsHoliday_x']).rename(columns={'IsHoliday_y': 'IsHoliday'})
df = df[['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
         'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'Type', 'MarkDown1',
         'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].sort_values(by=['Store', 'Dept', 'Date'])

test_df = pd.merge(test, features, on=['Store', 'Date'], how='left')
test_df = pd.merge(test_df, stores, on='Store', how='left')
test_df = test_df.drop(columns=['IsHoliday_x']).rename(columns={'IsHoliday_y': 'IsHoliday'})
test_df = test_df[['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature',
                   'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'Type',
                   'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].sort_values(by=['Store', 'Dept', 'Date'])

# Enhanced preprocessing
def enhance_preprocessing(df, test_df):
    """Enhanced preprocessing with additional features"""
    # Create IsHolidayWeight
    df['IsHolidayWeight'] = df['IsHoliday'].map({True: 5, False: 1}).astype('int8')
    test_df['IsHolidayWeight'] = test_df['IsHoliday'].map({True: 5, False: 1}).astype('int8')

    # Create store type dummy variables
    df = pd.get_dummies(df, columns=['Type'], prefix='Type')
    test_df = pd.get_dummies(test_df, columns=['Type'], prefix='Type')
    for col in ['Type_A', 'Type_B', 'Type_C']:
        if col not in df.columns:
            df[col] = 0
        if col not in test_df.columns:
            test_df[col] = 0
        df[col] = df[col].astype('int8')
        test_df[col] = test_df[col].astype('int8')

    # Store-level features
    store_stats = df.groupby('Store').agg({
        'Weekly_Sales': ['mean', 'std'],
        'Size': 'first'
    }).round(2)
    store_stats.columns = ['Store_Sales_Mean', 'Store_Sales_Std', 'Store_Size']
    store_stats['Store_Sales_Std'] = store_stats['Store_Sales_Std'].fillna(0).astype('float32')
    store_stats['Store_Sales_Mean'] = store_stats['Store_Sales_Mean'].astype('float32')
    store_stats['Store_Size'] = store_stats['Store_Size'].astype('int32')
    df = df.merge(store_stats, left_on='Store', right_index=True, how='left')
    test_df = test_df.merge(store_stats, left_on='Store', right_index=True, how='left')

    # Department-level features
    dept_stats = df.groupby('Dept').agg({
        'Weekly_Sales': ['mean', 'std']
    }).round(2)
    dept_stats.columns = ['Dept_Sales_Mean', 'Dept_Sales_Std']
    dept_stats['Dept_Sales_Std'] = dept_stats['Dept_Sales_Std'].fillna(0).astype('float32')
    dept_stats['Dept_Sales_Mean'] = dept_stats['Dept_Sales_Mean'].astype('float32')
    df = df.merge(dept_stats, left_on='Dept', right_index=True, how='left')
    test_df = test_df.merge(dept_stats, left_on='Dept', right_index=True, how='left')

    # Vectorized holiday features
    holiday_dates = {
        'SuperBowl': ['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'],
        'LaborDay': ['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'],
        'Thanksgiving': ['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'],
        'Christmas': ['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']
    }
    for holiday, dates in holiday_dates.items():
        holiday_pd_dates = pd.to_datetime(dates)
        df[f'{holiday}_Week'] = 0.0
        test_df[f'{holiday}_Week'] = 0.0
        for date in holiday_pd_dates:
            df[f'{holiday}_Week'] = np.where(df['Date'] == date, 1.0, df[f'{holiday}_Week'])
            test_df[f'{holiday}_Week'] = np.where(test_df['Date'] == date, 1.0, test_df[f'{holiday}_Week'])
            for i in range(1, 4):
                before_date = date - pd.Timedelta(weeks=i)
                after_date = date + pd.Timedelta(weeks=i)
                weight = 1.0 / (i + 1)
                df[f'{holiday}_Week'] = np.where(df['Date'].isin([before_date, after_date]), weight, df[f'{holiday}_Week'])
                test_df[f'{holiday}_Week'] = np.where(test_df['Date'].isin([before_date, after_date]), weight, test_df[f'{holiday}_Week'])
        df[f'{holiday}_Week'] = df[f'{holiday}_Week'].astype('float32')
        test_df[f'{holiday}_Week'] = test_df[f'{holiday}_Week'].astype('float32')

    # Time-based and cyclical features
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype('int8')
    df['Month'] = df['Date'].dt.month.astype('int8')
    df['Year'] = (df['Date'].dt.year - df['Date'].dt.year.min()).astype('int8')
    df['Week_sin'] = np.sin(2 * np.pi * df['WeekOfYear'] / 52).astype('float32')
    df['Week_cos'] = np.cos(2 * np.pi * df['WeekOfYear'] / 52).astype('float32')
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12).astype('float32')
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12).astype('float32')

    test_df['WeekOfYear'] = test_df['Date'].dt.isocalendar().week.astype('int8')
    test_df['Month'] = test_df['Date'].dt.month.astype('int8')
    test_df['Year'] = (test_df['Date'].dt.year - df['Date'].dt.year.min()).astype('int8')
    test_df['Week_sin'] = np.sin(2 * np.pi * test_df['WeekOfYear'] / 52).astype('float32')
    test_df['Week_cos'] = np.cos(2 * np.pi * test_df['WeekOfYear'] / 52).astype('float32')
    test_df['Month_sin'] = np.sin(2 * np.pi * test_df['Month'] / 12).astype('float32')
    test_df['Month_cos'] = np.cos(2 * np.pi * test_df['Month'] / 12).astype('float32')

    # Handle missing values
    for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']:
        df[col] = df.groupby(['Store', 'Month'])[col].transform('mean').fillna(df[col].mean()).astype('float32')
        test_df[col] = test_df.groupby(['Store', 'Month'])[col].transform('mean').fillna(test_df[col].mean()).astype('float32')

    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        df[col] = df[col].fillna(0).astype('float32')
        test_df[col] = test_df[col].fillna(0).astype('float32')

    # Interaction and lagged features
    df['Total_Markdown'] = df[markdown_cols].sum(axis=1).astype('float32')
    test_df['Total_Markdown'] = test_df[markdown_cols].sum(axis=1).astype('float32')
    df['Lagged_Sales'] = df.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).astype('float32')
    df['Lagged_Sales'] = df['Lagged_Sales'].fillna(df.groupby(['Store', 'Dept'])['Weekly_Sales'].transform('mean')).astype('float32')
    test_df['Lagged_Sales'] = df.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).reindex(test_df.index, fill_value=df['Weekly_Sales'].mean()).astype('float32')

    return df, test_df

# Apply enhanced preprocessing
df, test_df = enhance_preprocessing(df, test_df)

# Compute department-level and store-level means for fallback
dept_means = df.groupby(['Dept', 'IsHoliday'])['Weekly_Sales'].mean().unstack().fillna(df['Weekly_Sales'].mean())
store_means = df.groupby(['Store', 'IsHoliday'])['Weekly_Sales'].mean().unstack().fillna(df['Weekly_Sales'].mean())

In [19]:
def calculate_wmae(y_true, y_pred, weights):
    """Calculate Weighted Mean Absolute Error"""
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

def print_memory_usage():
    """Monitor memory usage"""
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"Memory Usage: {mem_info.rss / 1024**2:.2f} MB")
    wandb.log({"memory_usage_mb": mem_info.rss / 1024**2})

In [ ]:
# ✅ Enhanced forecaster with SARIMAX and Random Forest fallback
class EnhancedWalmartForecaster:
    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.store_dept_stats = {}
        self.sarimax_features = [
            'Total_Markdown', 'Lagged_Sales', 'SuperBowl_Week', 'Thanksgiving_Week',
            'Christmas_Week', 'Week_sin', 'Week_cos'
        ]
        self.fallback_features = [
            'Total_Markdown', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
            'IsHoliday', 'WeekOfYear', 'Month', 'Size', 'Type_A', 'Type_B', 'Type_C',
            'Week_sin', 'Week_cos', 'Month_sin', 'Month_cos', 'Store_Sales_Mean',
            'Dept_Sales_Mean', 'Lagged_Sales'
        ]

    def create_lag_features(self, df, target_col='Weekly_Sales', lags=[1, 2]):
        """Create lag features for time series"""
        df_lag = df.copy()
        for lag in lags:
            df_lag[f'{target_col}_lag_{lag}'] = df_lag.groupby(['Store', 'Dept'])[target_col].shift(lag).astype('float32')
        return df_lag

    def create_rolling_features(self, df, target_col='Weekly_Sales', windows=[4]):
        """Create rolling window features"""
        df_roll = df.copy()
        for window in windows:
            df_roll[f'{target_col}_roll_mean_{window}'] = (
                df_roll.groupby(['Store', 'Dept'])[target_col]
                .rolling(window=window, min_periods=1).mean()
                .reset_index(0, drop=True)
            ).astype('float32')
        return df_roll

    def fit_store_dept_model(self, store, dept, train_data):
        """Fit model for specific store-department combination"""
        try:
            store_dept_data = train_data[
                (train_data['Store'] == store) & (train_data['Dept'] == dept)
            ].copy().sort_values('Date')

            if len(store_dept_data) < 15:
                return self._fit_statistical_fallback(store, dept, store_dept_data)

            self.store_dept_stats[(store, dept)] = {
                'mean': store_dept_data['Weekly_Sales'].mean(),
                'std': store_dept_data['Weekly_Sales'].std(),
                'holiday_mean': store_dept_data[store_dept_data['IsHoliday']]['Weekly_Sales'].mean(),
                'non_holiday_mean': store_dept_data[~store_dept_data['IsHoliday']]['Weekly_Sales'].mean(),
                'count': len(store_dept_data)
            }

            if store_dept_data['Weekly_Sales'].std() < 1.0:
                return self._fit_statistical_fallback(store, dept, store_dept_data)

            if len(store_dept_data) >= 50 and store_dept_data['Weekly_Sales'].std() > 1000:
                success = self._try_sarimax(store, dept, store_dept_data)
                if success:
                    return True

            return self._fit_random_forest_fallback(store, dept, store_dept_data)

        except Exception as e:
            print(f"Error fitting model for Store {store}, Dept {dept}: {str(e)}")
            return self._fit_statistical_fallback(store, dept, store_dept_data)

    def _try_sarimax(self, store, dept, store_dept_data):
        """Try to fit SARIMAX model with simplified parameters"""
        try:
            y = store_dept_data['Weekly_Sales'].values
            exog_data = store_dept_data[self.sarimax_features].copy()
            for col in exog_data.columns:
                exog_data[col] = exog_data[col].fillna(exog_data[col].mean()).astype('float32')

            scaler = StandardScaler()
            exog_scaled = scaler.fit_transform(exog_data)
            self.scalers[(store, dept)] = scaler

            split_idx = int(len(y) * 0.8)
            train_y = y[:split_idx]
            train_exog = exog_scaled[:split_idx]
            val_y = y[split_idx:]
            val_exog = exog_scaled[split_idx:]
            val_weights = store_dept_data['IsHolidayWeight'].iloc[split_idx:].values

            model = SARIMAX(
                endog=train_y,
                exog=train_exog,
                order=(1, 0, 0),
                seasonal_order=(0, 0, 0, 52),
                enforce_stationarity=False,
                enforce_invertibility=False,
                simple_differencing=True
            )
            fitted_model = model.fit(disp=False, maxiter=50, method='lbfgs', gtol=1e-6)

            pred = fitted_model.forecast(steps=len(val_y), exog=val_exog)
            pred = np.maximum(pred, 0)
            wmae = calculate_wmae(val_y, pred, val_weights)

            if wmae < 10000:
                final_model = SARIMAX(
                    endog=y,
                    exog=exog_scaled,
                    order=(1, 0, 1),
                    seasonal_order=(0, 1, 1, 52),
                    enforce_stationarity=False,
                    enforce_invertibility=False,
                    simple_differencing=True
                )
                fitted_final = final_model.fit(disp=False, maxiter=50)
                self.models[(store, dept)] = {
                    'type': 'sarimax',
                    'model': fitted_final,
                    'cv_score': wmae
                }
                return True
            return False

        except Exception as e:
            return False

    def _fit_random_forest_fallback(self, store, dept, store_dept_data):
        """Fit Random Forest fallback model"""
        try:
            enhanced_data = self.create_lag_features(store_dept_data, lags=[1, 2])
            enhanced_data = self.create_rolling_features(enhanced_data, windows=[4])
            enhanced_data = enhanced_data.dropna()

            if len(enhanced_data) < 8:
                return self._fit_statistical_fallback(store, dept, store_dept_data)

            available_features = [col for col in self.fallback_features if col in enhanced_data.columns]
            lag_features = [col for col in enhanced_data.columns if 'lag_' in col or 'roll_' in col]
            all_features = available_features + lag_features

            X = enhanced_data[all_features].fillna(0)
            y = enhanced_data['Weekly_Sales'].values

            split_idx = int(len(X) * 0.8)
            X_train, X_val = X.iloc[:split_idx], X.iloc[split_idx:]
            y_train, y_val = y[:split_idx], y[split_idx:]

            if len(X_train) < 5:
                return self._fit_statistical_fallback(store, dept, store_dept_data)

            rf_model = RandomForestRegressor(
                n_estimators=50, max_depth=6, min_samples_split=5,
                min_samples_leaf=3, random_state=42, n_jobs=1, max_features='sqrt'
            )
            rf_model.fit(X_train, y_train)
            self.models[(store, dept)] = {
                'type': 'random_forest',
                'model': rf_model,
                'feature_cols': all_features
            }
            return True

        except Exception as e:
            return self._fit_statistical_fallback(store, dept, store_dept_data)

    def _fit_statistical_fallback(self, store, dept, store_dept_data):
        """Fit statistical fallback model"""
        try:
            stats = self.store_dept_stats.get((store, dept), {})
            holiday_mean = store_dept_data[store_dept_data['IsHoliday']]['Weekly_Sales'].mean()
            non_holiday_mean = store_dept_data[~store_dept_data['IsHoliday']]['Weekly_Sales'].mean()
            overall_mean = store_dept_data['Weekly_Sales'].mean()

            if pd.isna(holiday_mean):
                holiday_mean = overall_mean
            if pd.isna(non_holiday_mean):
                non_holiday_mean = overall_mean
            if pd.isna(overall_mean):
                overall_mean = 1000

            self.models[(store, dept)] = {
                'type': 'statistical',
                'holiday_mean': holiday_mean,
                'non_holiday_mean': non_holiday_mean,
                'overall_mean': overall_mean
            }
            return True

        except Exception as e:
            self.models[(store, dept)] = {
                'type': 'statistical',
                'holiday_mean': 1000,
                'non_holiday_mean': 1000,
                'overall_mean': 1000
            }
            return True

    def predict_store_dept(self, store, dept, test_group):
        """Make predictions for specific store-department"""
        try:
            if (store, dept) not in self.models:
                dept = test_group['Dept'].iloc[0] if not test_group.empty else 1
                is_holiday = test_group['IsHoliday'].iloc[0] if not test_group.empty else False
                store = test_group['Store'].iloc[0] if not test_group.empty else 1
                dept_mean = dept_means.loc[dept, is_holiday] if dept in dept_means.index else 1000
                store_mean = store_means.loc[store, is_holiday] if store in store_means.index else 1000
                pred_value = 0.6 * dept_mean + 0.4 * store_mean
                return np.full(len(test_group), pred_value)

            model_info = self.models[(store, dept)]
            if model_info['type'] == 'sarimax':
                fitted_model = model_info['model']
                scaler = self.scalers[(store, dept)]
                exog_data = test_group[self.sarimax_features].copy()
                for col in exog_data.columns:
                    exog_data[col] = exog_data[col].fillna(exog_data[col].mean()).astype('float32')
                exog_scaled = scaler.transform(exog_data)
                predictions = fitted_model.forecast(steps=len(test_group), exog=exog_scaled)
                return np.maximum(predictions, 0)

            elif model_info['type'] == 'random_forest':
                rf_model = model_info['model']
                feature_cols = model_info['feature_cols']
                test_group_enhanced = self.create_lag_features(test_group, lags=[1, 2])
                test_group_enhanced = self.create_rolling_features(test_group_enhanced, windows=[4])
                available_features = [col for col in feature_cols if col in test_group_enhanced.columns]
                X_test = test_group_enhanced[available_features].fillna(0)
                predictions = rf_model.predict(X_test)
                return np.maximum(predictions, 0)

            else:
                holiday_mean = model_info['holiday_mean']
                non_holiday_mean = model_info['non_holiday_mean']
                predictions = np.where(test_group['IsHoliday'], holiday_mean, non_holiday_mean)
                return np.maximum(predictions, 0)

        except Exception as e:
            print(f"Error predicting for Store {store}, Dept {dept}: {str(e)}")
            dept = test_group['Dept'].iloc[0] if not test_group.empty else 1
            is_holiday = test_group['IsHoliday'].iloc[0] if not test_group.empty else False
            store = test_group['Store'].iloc[0] if not test_group.empty else 1
            dept_mean = dept_means.loc[dept, is_holiday] if dept in dept_means.index else 1000
            store_mean = store_means.loc[store, is_holiday] if store in store_means.index else 1000
            pred_value = 0.6 * dept_mean + 0.4 * store_mean
            return np.full(len(test_group), pred_value)

def train_and_predict_enhanced_model(train_df, test_df):
    """Main function to train enhanced model and generate predictions"""
    print("🚀 Starting Enhanced Walmart Forecasting...")
    print("📊 Enhancing preprocessing...")
    wandb.init(
        project="walmart-sales-forecasting",
        name="sarimax_rf_hybrid",
        config={
            "model": "SARIMAX_RF_Ensemble",
            "features": "enhanced_with_store_dept_stats_lagged",
            "fallback_strategy": "sarimax_rf_statistical",
            "sarimax_order": (1, 0, 1),
            "sarimax_seasonal_order": (0, 1, 1, 52),
            "rf_n_estimators": 50,
            "rf_max_depth": 6
        }
    )
    forecaster = EnhancedWalmartForecaster()
    store_dept_combinations = train_df[['Store', 'Dept']].drop_duplicates()
    print(f"📈 Training models for {len(store_dept_combinations)} store-department combinations...")

    model_type_counts = {'sarimax': 0, 'random_forest': 0, 'statistical': 0}
    start_time = time.time()
    batch_size = 500
    batches = [store_dept_combinations[i:i+batch_size] for i in range(0, len(store_dept_combinations), batch_size)]

    for batch_idx, batch in enumerate(batches):
        print(f"Processing batch {batch_idx+1}/{len(batches)}...")
        print_memory_usage()
        for idx, (_, row) in enumerate(batch.iterrows()):
            store, dept = row['Store'], row['Dept']
            if idx % 50 == 0:
                elapsed = time.time() - start_time
                rate = (batch_idx * batch_size + idx) / elapsed if elapsed > 0 else 0
                eta = (len(store_dept_combinations) - (batch_idx * batch_size + idx)) / rate if rate > 0 else 0
                print(f"Progress: {batch_idx * batch_size + idx}/{len(store_dept_combinations)} "
                      f"({(batch_idx * batch_size + idx)/len(store_dept_combinations)*100:.1f}%) | "
                      f"Rate: {rate:.1f}/sec | ETA: {eta/60:.1f} min | "
                      f"SARIMAX: {model_type_counts['sarimax']}, RF: {model_type_counts['random_forest']}, "
                      f"Statistical: {model_type_counts['statistical']}")
            success = forecaster.fit_store_dept_model(store, dept, train_df)
            if (store, dept) in forecaster.models:
                model_type = forecaster.models[(store, dept)]['type']
                model_type_counts[model_type] += 1
        pd.DataFrame([(k, v) for k, v in forecaster.models.items()]).to_pickle(f'/content/batch_models_{batch_idx}.pkl')
        gc.collect()

    training_time = time.time() - start_time
    print(f"✅ Training completed in {training_time/60:.1f} minutes")
    print(f"📊 Model distribution: {model_type_counts}")

    print("🔮 Generating predictions...")
    predictions = []
    test_groups = test_df.groupby(['Store', 'Dept'])
    missing_store_depts = set()

    for (store, dept), test_group in test_groups:
        test_group_sorted = test_group.sort_values('Date')
        group_predictions = forecaster.predict_store_dept(store, dept, test_group_sorted)
        for idx, (_, test_row) in enumerate(test_group_sorted.iterrows()):
            test_id = f"{store}_{dept}_{test_row['Date'].strftime('%Y-%m-%d')}"
            predictions.append({
                'Id': test_id,
                'Weekly_Sales': group_predictions[idx]
            })

    existing_ids = {pred['Id'] for pred in predictions}
    overall_mean = train_df['Weekly_Sales'].mean()

    for _, test_row in test_df.iterrows():
        test_id = f"{test_row['Store']}_{test_row['Dept']}_{test_row['Date'].strftime('%Y-%m-%d')}"
        if test_id not in existing_ids:
            dept = test_row['Dept']
            store = test_row['Store']
            is_holiday = test_row['IsHoliday']
            dept_mean = dept_means.loc[dept, is_holiday] if dept in dept_means.index else overall_mean
            store_mean = store_means.loc[store, is_holiday] if store in store_means.index else overall_mean
            pred_value = 0.6 * dept_mean + 0.4 * store_mean
            predictions.append({
                'Id': test_id,
                'Weekly_Sales': max(0, pred_value)
            })
            missing_store_depts.add((store, dept))

    submission_df = pd.DataFrame(predictions)
    pred_values = submission_df['Weekly_Sales'].values
    print(f"\n📈 PREDICTION QUALITY CHECKS:")
    print(f"Total predictions: {len(submission_df)}")
    print(f"Expected predictions: {len(test_df)}")
    print(f"Min prediction: {pred_values.min():.2f}")
    print(f"Max prediction: {pred_values.max():.2f}")
    print(f"Mean prediction: {pred_values.mean():.2f}")
    print(f"Negative predictions: {(pred_values < 0).sum()}")

    wandb.log({
        "training_time_minutes": training_time/60,
        "sarimax_models": model_type_counts['sarimax'],
        "rf_models": model_type_counts['random_forest'],
        "statistical_models": model_type_counts['statistical'],
        "total_predictions": len(submission_df),
        "mean_prediction": pred_values.mean(),
        "prediction_std": pred_values.std(),
        "missing_store_depts": len(missing_store_depts)
    })
    return submission_df, forecaster

# Run enhanced model
print("🔄 Starting enhanced model training...")
enhanced_submission, trained_forecaster = train_and_predict_enhanced_model(df, test_df)
enhanced_submission.to_csv('/content/sarimax_rf_hybrid_submission.csv', index=False)
print("✅ Submission saved to /content/sarimax_rf_hybrid_submission.csv")
wandb.finish()